In [1]:
# system
import os, sys, glob, re, itertools, collections, requests
import multiprocessing # parallelise list comprehensions
from pathlib import Path
# pyscience imports
import numpy as np
import pandas as pd
import janitor
import pandas_flavor as pf

import statsmodels.api as sm
import statsmodels.formula.api as smf

# viz
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *
font = {'family' : 'IBM Plex Sans',
               'weight' : 'normal',
               'size'   : 10}
plt.rc('font', **font)
plt.rcParams['figure.figsize'] = (10, 10)
matplotlib.style.use(['seaborn-talk', 'seaborn-ticks', 'seaborn-whitegrid'])
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import locale
from locale import atof
locale.setlocale(locale.LC_NUMERIC, '')

'en_GB.UTF-8'

In [3]:
root = Path('/home/alal/Dropbox/1_Research/LongSHOT')
inp = root/'input'
tmp = root/'tmp'

# Function to slice and clean data

In [4]:
def make_header(df, stub):
    new_header = [stub + "__" + x.replace(" ", "_").replace("-","_") for x in df.iloc[0]]
    df = df[1:]
    df.columns = new_header
    return df

In [5]:
def data_cleaner(fn, splitu = False):
    name = fn.split('/')[-1]
    county_name, lea_name = name.split('-')[0].strip().replace("Co.", "") , name.split('-')[1].strip().split('.')[0]
    if splitu:
        county_name = name.split('-')[0].strip().replace("Co.", "") 
        lea_name = name.split('-')[1].strip().split('.')[0].split('_')[0]   
    df = pd.read_csv(fn)
    # slice and transpose rows from raw messy dataframe
    v_crimes   = make_header(df.loc[1:6].T, "vio_crimes_n")
    v_weapons  = make_header(df.loc[8:11].T, "vio_crimes_weap")
    v_location = make_header(df.loc[13:20].T, "vio_crimes_loc")
    p_crimes   = make_header(df.loc[26:27].T, "prop_crimes_n")
    p_type     = make_header(df.loc[46:54].T, "prop_crimes_type")
    # merge them by year, add county, LEA, and year 
    outdf = (pd.concat([v_crimes, v_weapons, v_location, p_crimes, p_type], axis = 1))
    outdf['county'] = county_name
    outdf['LEA'] = lea_name
    rest = [x for x in outdf.columns if x not in ['county', 'LEA']]
    outdf = outdf[['county', 'LEA'] + rest].reset_index().rename({'index':'year'}, axis = 1)
    return outdf

# 2010 - 2019

In [6]:
crime_root = root/"scrape/csvs/10_19"
%cd $crime_root

/home/alal/Dropbox/1_Research/LongSHOT/scrape/csvs/10_19


In [7]:
files = glob.glob("*.csv")
files.sort()
files[:5]

['Alameda Co. - Alameda BART.csv',
 "Alameda Co. - Alameda Co. Sheriff's Department.csv",
 'Alameda Co. - Alameda E. Bay Mun. Util. Dist..csv',
 'Alameda Co. - Alameda E. Bay Reg. Park Dist..csv',
 'Alameda Co. - Alameda SP RR.csv']

## Raw data

In [8]:
df = pd.read_csv("Los Angeles Co. - Los Angeles.csv")
df

,,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Crimes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Violent Crimes,"21,484","20,045","18,547","16,524","19,171","25,156","28,817","30,507","30,126","29,400"
2,Homicide,293,297,299,251,260,282,293,281,258,258
3,Rape (Forcible Rape prior to 2014),923,828,936,764,"1,126","2,209","2,343","2,455","2,528","2,274"
4,Rape,793,741,816,687,"1,016","2,065","2,218","2,337","2,443","2,172"
...,...,...,...,...,...,...,...,...,...,...,...
59,Under $50,"12,058","11,140","11,483","11,097","11,240","13,412","14,731","16,190","16,527","15,702"
60,Arson,"1,341","1,376","1,386","1,430","1,137","1,131","1,241","1,414","1,654","1,672"
61,Structural Property,232,193,191,205,173,199,217,264,246,269
62,Mobile Property,359,308,248,217,195,208,170,169,157,112


## dry run on LA data

In [9]:
odf = data_cleaner("Los Angeles Co. - Los Angeles.csv")
odf

,year,county,LEA,vio_crimes_n__Violent_Crimes,vio_crimes_n__Homicide,vio_crimes_n__Rape_(Forcible_Rape_prior_to_2014),vio_crimes_n__Rape,vio_crimes_n__Attempted_Rape,vio_crimes_n__Robbery,vio_crimes_weap__Firearm,...,prop_crimes_n__Burglary,prop_crimes_type__Pocket_Picking,prop_crimes_type__Purse_Snatching,prop_crimes_type__Shoplifting,prop_crimes_type__From_Motor_Vehicle,prop_crimes_type__Motor_Vehicle_Accessories,prop_crimes_type__Bicycles,prop_crimes_type__From_Building,prop_crimes_type__Coin_Operated_Machine,prop_crimes_type__Other
0,2010,Los Angeles,Los Angeles,"21,484",293,923,793,130,"10,924","3,577",...,"17,410",25,206,"3,563","21,929","5,806",160,"17,389",11,"6,159"
1,2011,Los Angeles,Los Angeles,"20,045",297,828,741,87,"10,077","2,887",...,"17,264",28,129,"3,664","21,208","4,414",254,"17,048",21,"6,703"
2,2012,Los Angeles,Los Angeles,"18,547",299,936,816,120,"8,983","2,363",...,"16,388",39,97,"3,969","21,891","4,474",619,"17,772",22,"7,123"
3,2013,Los Angeles,Los Angeles,"16,524",251,764,687,77,"7,885","2,220",...,"15,728",43,93,"3,817","21,591","4,662",799,"17,449",22,"7,258"
4,2014,Los Angeles,Los Angeles,"19,171",260,"1,126","1,016",110,"7,949","2,042",...,"15,070",45,102,"4,875","13,371","10,885","1,146","23,811",10,36
5,2015,Los Angeles,Los Angeles,"25,156",282,"2,209","2,065",144,"8,952","2,294",...,"16,160",134,147,"6,447","14,701","13,567","1,716","18,388",54,"6,037"
6,2016,Los Angeles,Los Angeles,"28,817",293,"2,343","2,218",125,"10,307","2,816",...,"15,821",148,126,"7,146","17,070","14,342","2,335","17,592",51,"5,929"
7,2017,Los Angeles,Los Angeles,"30,507",281,"2,455","2,337",118,"10,814","2,804",...,"16,668",205,141,"6,476","18,399","14,316","2,366","17,734",48,"6,072"
8,2018,Los Angeles,Los Angeles,"30,126",258,"2,528","2,443",85,"10,327","2,536",...,"15,988",170,107,"6,667","18,316","14,741","2,482","18,966",31,"6,483"
9,2019,Los Angeles,Los Angeles,"29,400",258,"2,274","2,172",102,"9,652","2,308",...,"13,809",212,79,"6,776","16,959","14,372","2,438","18,541",35,"6,841"


numbers are still in string format at the moment - will clean at once with final dataset to avoid accounting for very small towns with no commas.

## Apply to all places and stack

In [10]:
%%time 
crime_cleaned = [data_cleaner(f) for f in files]

CPU times: user 7.83 s, sys: 24.8 ms, total: 7.86 s
Wall time: 7.86 s


In [11]:
all = pd.concat(crime_cleaned, axis = 0).convert_dtypes()
all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8700 entries, 0 to 9
Data columns (total 32 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   year                                              8700 non-null   string
 1   county                                            8700 non-null   string
 2   LEA                                               8700 non-null   string
 3   vio_crimes_n__Violent_Crimes                      8700 non-null   object
 4   vio_crimes_n__Homicide                            8700 non-null   object
 5   vio_crimes_n__Rape_(Forcible_Rape_prior_to_2014)  8700 non-null   object
 6   vio_crimes_n__Rape                                8700 non-null   object
 7   vio_crimes_n__Attempted_Rape                      8700 non-null   object
 8   vio_crimes_n__Robbery                             8700 non-null   object
 9   vio_crimes_weap__Firearm         

In [12]:
num_cols = all.columns[3:]
all[num_cols] =  (all[num_cols]
                  .astype('str') # convert 'objects' to string - pandas is too flexible 
                  .apply(lambda x: x.replace("--", "0")) # replace pesky -- empty rows
                  .applymap(atof) # convert to numeric by parsing commas 
                 )

## sanity check

In [13]:
ladat = all.query("LEA == 'Los Angeles'")
ladat.head()
ladat.describe()

,year,county,LEA,vio_crimes_n__Violent_Crimes,vio_crimes_n__Homicide,vio_crimes_n__Rape_(Forcible_Rape_prior_to_2014),vio_crimes_n__Rape,vio_crimes_n__Attempted_Rape,vio_crimes_n__Robbery,vio_crimes_weap__Firearm,...,prop_crimes_n__Burglary,prop_crimes_type__Pocket_Picking,prop_crimes_type__Purse_Snatching,prop_crimes_type__Shoplifting,prop_crimes_type__From_Motor_Vehicle,prop_crimes_type__Motor_Vehicle_Accessories,prop_crimes_type__Bicycles,prop_crimes_type__From_Building,prop_crimes_type__Coin_Operated_Machine,prop_crimes_type__Other
0,2010,Los Angeles,Los Angeles,21484.0,293.0,923.0,793.0,130.0,10924.0,3577.0,...,17410.0,25.0,206.0,3563.0,21929.0,5806.0,160.0,17389.0,11.0,6159.0
1,2011,Los Angeles,Los Angeles,20045.0,297.0,828.0,741.0,87.0,10077.0,2887.0,...,17264.0,28.0,129.0,3664.0,21208.0,4414.0,254.0,17048.0,21.0,6703.0
2,2012,Los Angeles,Los Angeles,18547.0,299.0,936.0,816.0,120.0,8983.0,2363.0,...,16388.0,39.0,97.0,3969.0,21891.0,4474.0,619.0,17772.0,22.0,7123.0
3,2013,Los Angeles,Los Angeles,16524.0,251.0,764.0,687.0,77.0,7885.0,2220.0,...,15728.0,43.0,93.0,3817.0,21591.0,4662.0,799.0,17449.0,22.0,7258.0
4,2014,Los Angeles,Los Angeles,19171.0,260.0,1126.0,1016.0,110.0,7949.0,2042.0,...,15070.0,45.0,102.0,4875.0,13371.0,10885.0,1146.0,23811.0,10.0,36.0


,vio_crimes_n__Violent_Crimes,vio_crimes_n__Homicide,vio_crimes_n__Rape_(Forcible_Rape_prior_to_2014),vio_crimes_n__Rape,vio_crimes_n__Attempted_Rape,vio_crimes_n__Robbery,vio_crimes_weap__Firearm,vio_crimes_weap__Knife_or_Cutting_Instrument,vio_crimes_weap__Other_Weapon,vio_crimes_weap__Strong_Arm,...,prop_crimes_n__Burglary,prop_crimes_type__Pocket_Picking,prop_crimes_type__Purse_Snatching,prop_crimes_type__Shoplifting,prop_crimes_type__From_Motor_Vehicle,prop_crimes_type__Motor_Vehicle_Accessories,prop_crimes_type__Bicycles,prop_crimes_type__From_Building,prop_crimes_type__Coin_Operated_Machine,prop_crimes_type__Other
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.00000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,23977.700000,277.200000,1638.600000,1528.800000,109.800000,9587.000000,2584.700000,1048.800000,1085.700000,4867.800000,...,16030.600000,104.900000,122.700000,5340.000000,18543.50000,10157.900000,1431.500000,18469.000000,30.500000,5864.100000
std,5420.958137,18.629725,772.679062,767.887984,21.734765,1103.092219,449.414199,122.432566,207.972781,496.971897,...,1052.528722,76.371097,36.578834,1490.067336,3079.10926,4715.855371,944.697277,1966.862194,16.105555,2100.263341
min,16524.000000,251.000000,764.000000,687.000000,77.000000,7885.000000,2042.000000,863.000000,857.000000,3945.000000,...,13809.000000,25.000000,79.000000,3563.000000,13371.00000,4414.000000,160.000000,17048.000000,10.000000,36.000000
25%,19389.500000,258.500000,926.250000,798.750000,90.750000,8959.750000,2297.500000,977.500000,901.500000,4677.500000,...,15751.250000,40.000000,98.250000,3855.000000,16986.75000,4948.000000,664.000000,17484.750000,21.250000,6045.750000
50%,23320.000000,281.500000,1667.500000,1540.500000,114.000000,9864.500000,2449.500000,1052.000000,1038.500000,5056.500000,...,16074.000000,89.500000,116.500000,5661.000000,18357.50000,12226.000000,1431.000000,17753.000000,26.500000,6321.000000
75%,29254.250000,293.000000,2325.750000,2206.500000,123.750000,10322.000000,2813.000000,1147.000000,1263.750000,5210.250000,...,16598.000000,164.500000,138.000000,6619.250000,21495.25000,14335.500000,2358.250000,18502.750000,44.750000,6806.500000
max,30507.000000,299.000000,2528.000000,2443.000000,144.000000,10924.000000,3577.000000,1214.000000,1394.000000,5448.000000,...,17410.000000,212.000000,206.000000,7146.000000,21929.00000,14741.000000,2482.000000,23811.000000,54.000000,7258.000000


In [14]:
all = all.convert_dtypes() # compress to smallest possible data types

In [15]:
all.to_csv(root/'output/openjustice_place_panel_2010_2019.csv', index = False)

# 2000-2009

In [16]:
crime_root = root/"scrape/csvs/00_09"
%cd $crime_root

/home/alal/Dropbox/1_Research/LongSHOT/scrape/csvs/00_09


In [17]:
files = glob.glob("*.csv")
files.sort()
files[:5]

['Alameda Co. - Alameda BART_00_09.csv',
 "Alameda Co. - Alameda Co. Sheriff's Department_00_09.csv",
 'Alameda Co. - Alameda E. Bay Mun. Util. Dist._00_09.csv',
 'Alameda Co. - Alameda E. Bay Reg. Park Dist._00_09.csv',
 'Alameda Co. - Alameda SP RR_00_09.csv']

In [18]:
%%time 
crime_cleaned = [data_cleaner(f, splitu = True) for f in files]

CPU times: user 8.5 s, sys: 21.4 ms, total: 8.53 s
Wall time: 8.52 s


In [19]:
all = pd.concat(crime_cleaned, axis = 0).convert_dtypes()
all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8700 entries, 0 to 9
Data columns (total 32 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   year                                              8700 non-null   string
 1   county                                            8700 non-null   string
 2   LEA                                               8700 non-null   string
 3   vio_crimes_n__Violent_Crimes                      8700 non-null   object
 4   vio_crimes_n__Homicide                            8700 non-null   object
 5   vio_crimes_n__Rape_(Forcible_Rape_prior_to_2014)  8700 non-null   object
 6   vio_crimes_n__Rape                                8700 non-null   object
 7   vio_crimes_n__Attempted_Rape                      8700 non-null   object
 8   vio_crimes_n__Robbery                             8700 non-null   object
 9   vio_crimes_weap__Firearm         

In [20]:
num_cols = all.columns[3:]
all[num_cols] =  (all[num_cols]
                  .astype('str') # convert 'objects' to string - pandas is too flexible 
                  .apply(lambda x: x.replace("--", "0")) # replace pesky -- empty rows
                  .applymap(atof) # convert to numeric by parsing commas 
                 )

In [21]:
all.head()

,year,county,LEA,vio_crimes_n__Violent_Crimes,vio_crimes_n__Homicide,vio_crimes_n__Rape_(Forcible_Rape_prior_to_2014),vio_crimes_n__Rape,vio_crimes_n__Attempted_Rape,vio_crimes_n__Robbery,vio_crimes_weap__Firearm,...,prop_crimes_n__Burglary,prop_crimes_type__Pocket_Picking,prop_crimes_type__Purse_Snatching,prop_crimes_type__Shoplifting,prop_crimes_type__From_Motor_Vehicle,prop_crimes_type__Motor_Vehicle_Accessories,prop_crimes_type__Bicycles,prop_crimes_type__From_Building,prop_crimes_type__Coin_Operated_Machine,prop_crimes_type__Other
0,2000,Alameda,Alameda BART,79.0,0.0,1.0,1.0,0.0,59.0,9.0,...,9.0,22.0,22.0,1.0,444.0,100.0,276.0,4.0,1.0,163.0
1,2001,Alameda,Alameda BART,105.0,0.0,1.0,1.0,0.0,81.0,11.0,...,13.0,15.0,21.0,0.0,672.0,89.0,282.0,2.0,1.0,176.0
2,2002,Alameda,Alameda BART,87.0,0.0,0.0,0.0,0.0,77.0,8.0,...,8.0,11.0,22.0,0.0,572.0,92.0,222.0,1.0,1.0,173.0
3,2003,Alameda,Alameda BART,100.0,0.0,0.0,0.0,0.0,90.0,8.0,...,6.0,2.0,29.0,0.0,589.0,90.0,294.0,2.0,0.0,204.0
4,2004,Alameda,Alameda BART,101.0,0.0,0.0,0.0,0.0,96.0,13.0,...,4.0,3.0,25.0,0.0,439.0,108.0,188.0,0.0,1.0,180.0


In [22]:
all = all.convert_dtypes() # compress to smallest possible data types

In [23]:
all.to_csv(root/'output/openjustice_place_panel_2000_2009.csv', index = False)

# 1990 -1999

In [24]:
crime_root = root/"scrape/csvs/90_99"
%cd $crime_root

/home/alal/Dropbox/1_Research/LongSHOT/scrape/csvs/90_99


In [25]:
files = glob.glob("*.csv")
files.sort()
files[:5]

['Alameda Co. - Alameda BART_90_99.csv',
 "Alameda Co. - Alameda Co. Sheriff's Department_90_99.csv",
 'Alameda Co. - Alameda E. Bay Mun. Util. Dist._90_99.csv',
 'Alameda Co. - Alameda E. Bay Reg. Park Dist._90_99.csv',
 'Alameda Co. - Alameda SP RR_90_99.csv']

In [27]:
%%time 
crime_cleaned = [data_cleaner(f, splitu = True) for f in files]

CPU times: user 8.64 s, sys: 83.1 ms, total: 8.72 s
Wall time: 8.72 s


In [28]:
all = pd.concat(crime_cleaned, axis = 0).convert_dtypes()
all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8700 entries, 0 to 9
Data columns (total 32 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   year                                              8700 non-null   string
 1   county                                            8700 non-null   string
 2   LEA                                               8700 non-null   string
 3   vio_crimes_n__Violent_Crimes                      8700 non-null   object
 4   vio_crimes_n__Homicide                            8700 non-null   object
 5   vio_crimes_n__Rape_(Forcible_Rape_prior_to_2014)  8700 non-null   object
 6   vio_crimes_n__Rape                                8700 non-null   object
 7   vio_crimes_n__Attempted_Rape                      8700 non-null   object
 8   vio_crimes_n__Robbery                             8700 non-null   object
 9   vio_crimes_weap__Firearm         

In [29]:
num_cols = all.columns[3:]
all[num_cols] =  (all[num_cols]
                  .astype('str') # convert 'objects' to string - pandas is too flexible 
                  .apply(lambda x: x.replace("--", "0")) # replace pesky -- empty rows
                  .applymap(atof) # convert to numeric by parsing commas 
                 )

In [30]:
all.head()

,year,county,LEA,vio_crimes_n__Violent_Crimes,vio_crimes_n__Homicide,vio_crimes_n__Rape_(Forcible_Rape_prior_to_2014),vio_crimes_n__Rape,vio_crimes_n__Attempted_Rape,vio_crimes_n__Robbery,vio_crimes_weap__Firearm,...,prop_crimes_n__Burglary,prop_crimes_type__Pocket_Picking,prop_crimes_type__Purse_Snatching,prop_crimes_type__Shoplifting,prop_crimes_type__From_Motor_Vehicle,prop_crimes_type__Motor_Vehicle_Accessories,prop_crimes_type__Bicycles,prop_crimes_type__From_Building,prop_crimes_type__Coin_Operated_Machine,prop_crimes_type__Other
0,1990,Alameda,Alameda BART,112.0,1.0,4.0,2.0,2.0,86.0,9.0,...,4.0,35.0,110.0,2.0,1027.0,138.0,187.0,4.0,1.0,127.0
1,1991,Alameda,Alameda BART,116.0,1.0,0.0,0.0,0.0,95.0,21.0,...,1.0,28.0,40.0,3.0,909.0,151.0,226.0,2.0,4.0,167.0
2,1992,Alameda,Alameda BART,99.0,1.0,1.0,1.0,0.0,77.0,19.0,...,7.0,27.0,33.0,5.0,835.0,155.0,187.0,1.0,0.0,207.0
3,1993,Alameda,Alameda BART,144.0,0.0,1.0,1.0,0.0,122.0,18.0,...,7.0,39.0,37.0,4.0,978.0,220.0,225.0,5.0,3.0,344.0
4,1994,Alameda,Alameda BART,138.0,0.0,0.0,0.0,0.0,115.0,18.0,...,10.0,6.0,50.0,2.0,935.0,151.0,193.0,3.0,1.0,319.0


In [31]:
all = all.convert_dtypes() # compress to smallest possible data types

In [33]:
all.to_csv(root/'output/openjustice_place_panel_1990_1999.csv', index = False)

## Stack

In [34]:
df1 = pd.read_csv(root/'output/openjustice_place_panel_1990_1999.csv')
df2 = pd.read_csv(root/'output/openjustice_place_panel_2000_2009.csv')
df3 = pd.read_csv(root/'output/openjustice_place_panel_2010_2019.csv')

In [35]:
df = pd.concat([df1, df2, df3])
df.sort_values(['county', 'LEA', 'year'], inplace = True)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26100 entries, 70 to 8699
Data columns (total 32 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   year                                              26100 non-null  int64 
 1   county                                            26100 non-null  object
 2   LEA                                               26100 non-null  object
 3   vio_crimes_n__Violent_Crimes                      26100 non-null  int64 
 4   vio_crimes_n__Homicide                            26100 non-null  int64 
 5   vio_crimes_n__Rape_(Forcible_Rape_prior_to_2014)  26100 non-null  int64 
 6   vio_crimes_n__Rape                                26100 non-null  int64 
 7   vio_crimes_n__Attempted_Rape                      26100 non-null  int64 
 8   vio_crimes_n__Robbery                             26100 non-null  int64 
 9   vio_crimes_weap__Firearm    

In [37]:
df.to_csv(root/'output/openjustice_place_panel_1990_2019.csv', index = False)